프롬프트 템플릿 : 언어 모델에 대한 프롬프트를 생성하기 위한 미리 정의된 레시피
- 지침, 예시, 특정 맥락 및 주어진 작업에 적합한 질문 포함될 수 있음.
- langchain은 다양한 언어 모델에서 쉽게 재사용될 수 있도록 모델에 독립적인 템플릿을 만듦
- 언어모델의 프롬프트 형태 : 문자열 또는 채팅 메시지

[PromptTemplate]
:문자열 프롬프트에 대한 템플릿 만듦
- 여러개의 변수를 둘 수도 있고, 변수가 없을 수도 있다.

In [2]:
from langchain.prompts import PromptTemplate # type: ignore

prompt_template = PromptTemplate.from_template(
    "tell me a {adjective} joke about {content}."
)

prompt_template.format(adjective="funny", content="chickens")

'tell me a funny joke about chickens.'

In [5]:
prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

[ChatPromptTemplate]
: chat model에게 주는 prompt로 챗메시지의 목록이다.
- 챗메시지 : role 매개변수와 내용으로 구성됨
- role(OpenAI Chat Completion API에서는) :  AI, human, system

In [7]:
from langchain.prompts import ChatPromptTemplate # type: ignore

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI bot. your name is {name}."),
        ("human" "Hello, how are you doing?"),
        ("ai", "I'm donig well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="what is your name?" )

messages

[SystemMessage(content='You are a helpful AI bot. your name is Bob.'),
 HumanMessage(content='humanHello, how are you doing?'),
 AIMessage(content="I'm donig well, thanks!"),
 HumanMessage(content='what is your name?')]

In [ ]:
- 형식화된 메시지를 langchain의 chat model 클래스(ChatOpenAI)에 piping하는 것은 OpenAI client를 직접 사용하는 것과 거의 같다. # type: ignore

In [9]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from langchain.llms.openai import OpenAI # type: ignore

client = OpenAI()

response = client.construct(
    model ="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":"You are a helpful AI bot. your name is Bob."},
        {"role":"human","content":"Hello, how are you doing?"},
        {"role":"assistant", "content":"I'm donig well, thanks!"},
        {"role":"human", "content":"What is your name"},
    ],
)
response

OpenAI(model_name='gpt-3.5-turbo', model='gpt-3.5-turbo', messages=[{'role': 'system', 'content': 'You are a helpful AI bot. your name is Bob.'}, {'role': 'human', 'content': 'Hello, how are you doing?'}, {'role': 'assistant', 'content': "I'm donig well, thanks!"}, {'role': 'human', 'content': 'What is your name'}])

- ChatPromptTemplat.from_messages()는 MessagePromptTemplate나 BaseMessage의 인스턴스를 전달할 수 있음.
- MessagePromptTemplat : AIMessagePromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
- ChatMessagePromptTemplate : 임의의 역할을 부여하고 싶을 때 사용

In [29]:
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate, SystemMessage # type: ignore

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-wrihe the user's text to sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

messages = chat_template.format_messages(text="I don't like eating tasty things")

messages




[SystemMessage(content="You are a helpful assistant that re-wrihe the user's text to sound more upbeat."),
 HumanMessage(content="I don't like eating tasty things")]

In [30]:
from langchain.prompts.chat import ChatMessagePromptTemplate # type: ignore

prompt = "May the {subject} be with you"

chat_message_prompt = ChatMessagePromptTemplate.from_template(
    role="Jedi", template=prompt
)

chat_message_prompt.format(subject="force")

ChatMessage(content='May the force be with you', role='Jedi')

[MessagesPlaceHolder] :
- 포맷팅 중에 랜더링할 메시지를 제어할 수 있음.
- 메시지 프롬프트 템플릿에 어떤 role을 사용할지 확실하지 않거나, 포매팅 중 메시지 목록을 삽입하려는 경우 유용

In [38]:
from langchain.prompts import ( # type: ignore
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [human_message_template,
     MessagesPlaceholder(variable_name="conversation") ]
)

chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain.schema.messages.AIMessage, langchain.schema.messages.HumanMessage, langchain.schema.messages.ChatMessage, langchain.schema.messages.SystemMessage, langchain.schema.messages.FunctionMessage]]}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='Summarize our conversation so far in {word_count} words.')), MessagesPlaceholder(variable_name='conversation')])

In [39]:
from langchain.schema.messages import AIMessage, HumanMessage # type: ignore

human_message = HumanMessage(content="What is the best way to learn programming?")

ai_message = AIMessage(
    content="""
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience
"""
)

chat_prompt.format_prompt(
    conversation=[human_message, ai_message], word_count="10"
).to_messages()

[HumanMessage(content='Summarize our conversation so far in 10 words.'),
 HumanMessage(content='What is the best way to learn programming?'),
 AIMessage(content='\n1. Choose a programming language: Decide on a programming language that you want to learn.\n\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.\n\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience\n')]

[LCEL] : LangChain Expression Language
- 